In [1]:
! pip install -q google-generativeai

In [5]:
import google.generativeai as genai
from pathlib import Path
from time import sleep

class GeminiInference():
  def __init__(self):
    from google.colab import userdata

    self.gemini_key = userdata.get('GEMINI')

    genai.configure(api_key=self.gemini_key)
    generation_config = {
        "temperature": 1,
        "top_p": 1,
        "top_k": 32,
        "max_output_tokens": 4096,
    }
    safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
    ]

    self.model = genai.GenerativeModel(model_name="gemini-1.5-flash",
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)


  def prompt(self, *args, **kwargs):
    """
    Write there custom way to prompt to model or just return the text

    """
    return " ".join([str(arg) for arg in args])

  def extract_target_answer(self, response):
    """
    Write there custom way to process the model output or just return the text

    """
    return response

  def get_response(self, input_text):
    prompt_parts = [
        input_text
    ]
    response = self.model.generate_content(prompt_parts)
    return response.text

  def __call__(self, *args, **kwargs):
    input_text = self.prompt(*args, **kwargs)

    output_text = self.get_response(input_text)

    return self.extract_target_answer(output_text)

In [8]:
import json

class GeminiForFeatureExtraction(GeminiInference):
  def __init__(self, ):
    super().__init__()

  def extract_target_answer(self, text):
        # Remove the Markdown formatting
    cleaned_text = text.strip("```json\n").strip("```")

    # Convert the cleaned JSON string to a Python dictionary
    try:
        data_dict = json.loads(cleaned_text)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

    return data_dict

  def prompt(self, text):
    prompt = (lambda text: '''You are not only language model, also talented mind created for help me to solve my tasks.
    You are the my helper for speeding up my HR tasks.
    I will show you text, you task is extract information in json format to like:

    {"Core Responsibilities": ["N/A"],
    "Required Skills": ["N/A"],
    "Educational Requirements": ["N/A"],
    "Experience Level": ["N/A"],
    "Preferred Qualifications": ["N/A"],
    "Compensation and Benefits": ["N/A"]\}
    ''' + f'''
    PLEASE GIVE ME ONLY THIS AS ANSWER.
    from vacancy or resume text
    My Text:
    "{text}"''')

    return prompt(text)

In [9]:
class GeminiForVacancyGeneration(GeminiInference):
  def __init__(self, ):
    super().__init__()

  def prompt(self,
             title: str,
             company: str,
             mode: str='online',
             skills = 'Python',
             salary = None,
             experince = None,
             ):

    vacancy_info = {
        "title": title,
        "salary":salary,
        "Employment mode": mode,
        "company": company,
        "experience": experince,
        "skills": skills
    }

    return ("""You are talented HR manager hired into my company, not language model.
Today your task is write detailed vacancies text (description, on russian language) which based on my information like:

{"title": None,
"salary":None,
"Employment mode": None,
"company": None,
"experience": None,
"skills": None
}
And example description have to look like:
"<START>
Центр организации дорожного движения активно участвует в формировании городской мобильности, разрабатывает и внедряет стратегии для более быстрого, комфортного и безопасного движения в . Стань частью и внеси свой вклад в развитие дорожно-транспортной инфраструктуры города! К : Обеспечение выполнения работ по содержанию зданий, сооружений и прилегающей территории в надлежащем состоянии, организация уборки, соблюдения чистоты во внутренних помещениях зданий и прилегающей территории Подготовка помещений к осенне-зимней эксплуатации Организация и контроль косметических ремонтов помещений, организация своевременного ремонта офисной мебели Обеспечение сохранности и содержание в исправном состоянии имущества, которое находится в зданиях Ведение учета наличия имущества, проведение периодического осмотра и составления актов на его списание Контроль выполнения правил противопожарной безопасности, знание и выполнение требований нормативных актов об охране труда и окружающей среды : Высшее/Среднее специальное образование Среднее техническое/Высшее техническое образование Опыт работы в данной или смежной областях со схожим функционалом от 1-го года Уверенный пользователь ПК, знание основных компьютерных программ (Word, Excel, Outlook, Internet) Ответственность, организованность, профессиональная грамотность и требовательность, внимательность к деталям в работе : Официальное трудоустройство в стабильном государственном учреждении График 5/2 с 8:00 до 17:00, в пятницу до 15:45 Система оплаты: ежемесячный стабильный доход + квартальные, годовые премии А ЕЩЕ МЫ ПРЕДЛАГАЕМ: Бесплатный проезд (Метро и МЦК) Профсоюзная организация: льготный ДМС, частичная компенсация затрат на санаторно-курортное лечение, оздоровительный отдых Корпоративные мероприятия для сотрудников Корпоративный тренажерный зал Для детей сотрудников: целевое обучение / прохождение практики в Транспортном комплексе Правительства Москвы Льготные ставки по кредитам в Сбербанке и ВТБ по нашим зарплатным проектам
<END>
"
PLEASE DONT FORGET ABOUT WRITING "<START>" TO THE START OF DESCRIPTION AND "<END>" TO THE END OF TEXT!

Lets start! There is the my vacancy info
""" + str(vacancy_info) + """
Please write vacancy description base on this information""")

  def extract_target_answer(self, text):
    return text.split("<START>")[1].split("<END>")[0]

In [13]:
vacancy_generator = GeminiForVacancyGeneration()

In [16]:
v = vacancy_generator(

    title = 'Python Developer',
    company = 'Style-Ex',
    mode = 'online',
    skills = 'Python',
    salary = "1020$",
    experince = "4 years",
)

print(v)



**Python  Developer**

**Style-Ex**  ищет талантливого и опытного Python разработчика для присоединения к нашей команде.

**Обязанности:**

* Разработка и поддержка высоконагруженных веб-приложений.
* Разработка новых функциональных модулей и API.
* Оптимизация производительности и масштабируемости кода.
* Тестирование и отладка кода.
* Участие в проектных совещаниях. 

**Требования:**

* Опыт работы с Python не менее 4 лет.
*  Хорошее знание Python, включая основные библиотеки  (Django/Flask, pandas, NumPy, etc).
*  Опыт работы с реляционными базами данных (MySQL, PostgreSQL).
*  Опыт работы с системами контроля версий (Git).
*  Умение работать в команде.

**Условия:** 

*  Удаленная работа.
*  Заработная плата: 1020$ (заработная плата  может быть скорректирована  в зависимости от опыта и навыков).
*  Возможность профессионального роста.

**О Style-Ex**:

**(Вставьте краткое описание вашей компании)**

Если вы хотите стать частью нашей команды и внести свой вклад в развитие Style-Ex